### 유튜브에서 특정 검색어의 컨텐츠 제목과 조회수 추출

In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pandas as pd
import numpy as np
import time

In [2]:
driver = wb.Chrome()
driver.maximize_window()
search = input("검색어 입력 >> ")
driver.get(f"https://www.youtube.com/results?search_query={search}")

titles = []
view_cnt = []

tit_list = []
cnt_list = []

target_cnt = 50
body = driver.find_element(By.TAG_NAME, "body")

# 페이지 스크롤
while len(titles) < target_cnt :
    body.send_keys(Keys.END)
    time.sleep(2)
    # 페이지 스크롤 후 elements 담기
    titles = driver.find_elements(By.CSS_SELECTOR, "#video-title > yt-formatted-string")
    view_cnt = driver.find_elements(By.CSS_SELECTOR, ".style-scope.ytd-video-renderer #metadata-line > span:nth-child(3)")
    
print(len(titles), len(view_cnt))

# 제목, 조회수 리스트에 추가
for i in tqdm(range(target_cnt)) :
    tit_list.append(titles[i].text)
    cnt_list.append(view_cnt[i].text)
    
print(len(tit_list), len(cnt_list))
# DataFrame 생성
yt_df = pd.DataFrame({"제목" : tit_list, "조회수" : cnt_list}, index = range(1, len(tit_list)+1))
yt_df.index.name = "No"
# 조회수 컬럼에서 '시청 중' 제외
yt_df = yt_df[~yt_df["조회수"].str.contains("시청 중")]
# 빈 문자열 -> NaN 변환
yt_df.replace("", np.nan, inplace = True)
# NaN 행 삭제
yt_df = yt_df.dropna()
yt_df

In [3]:
yt_df.to_csv("../data/youtube.csv", encoding = "utf-8")
driver.quit()